Phase 1
=======
In this Phase we try to find
- `Number of panels required`
- `Number of inverters required`
- `Arrangement of panels`
- `Cost of building the project`

The first step to build a power plant is to select
- `The Output/wattage it must produce`
- `Which panel to be used`
- `What inverter to be used`
<br>
<br>
The above elements are selected by user which is presented in a easy to understand UI.

<br>
After selecting the requirements of user. The Backend tries to produce the most cost efficient model to build a plant.
<br>
<br>
To kick it off, Lets start the project with finding the <b>number of panels for one Inverter

Imports
======

In [1]:
import math

Data
====

In [2]:
Panels = {
    'LG365N1C-N' : {
        'name' : 'LG NeON2',
        'dimensions' : (1700, 1016, 40),
        'length' : 1700,       #dimensions are measured in mm
        'width' : 1016,
        'height' : 40,
        'weight' : 18.0,        #weight in kg
        'cable_length' : '1000 mm x 2EA',
        'warranty' : 25,         #years
        'NMOT' : '42+-3',
        'Pmax' : -0.34,
        'Voc' : -0.26,
        'Isc' : 0.03,
        'Pmax' : 365,            #Watts
        'Vmpp' : 35.5,
        'Impp' : 10.30,
        'open_circuit_voltage' : 41.7,
        'short_circuit_voltage' : 10.88,
        'efficiency' : 21.1,
        'power_tolerance' : '0~+3',
        'price' : 360,           #Cost
        'cost' : 360,
    }
}

In [3]:
Panels = {
    'default' : {
        'name' : 'LG NeON2',
        'model' : 'LG365N1C-N',j
        'NOCT' : '42',
        'NOCT_Tolerance' : '3',
    }
}

SyntaxError: invalid syntax (<ipython-input-3-fab204e6108f>, line 5)

Symbols and meanings
===================

$T_{op}$ : T_op : Operating Temperature

Defaults
=======

In [4]:
T_amb = 28
T_amb_min = 5
T_amb_max = 38

NOCT = 47
G = 800

f_temp = 0.85


f_dirt = 0.95
f_mt = 0.95

V_mpp_stc = 36.6
V_oc_stc = 44.8

gamma_p = 0.4048
gamma_v_oc = 0.2931


V_inv_dc_min = 425
V_inv_dc_max = 975

I_sc_stc = 8.71

gamma_I_sc = 0.0442

I_inv_dc = 600


In [5]:
power_of_solar_panel = 36.6*8.71

Functions
========

In [6]:
def T_op(T_amb, NOCT, G):                       #Input expected in centigrade
    t_op = T_amb + ((NOCT - 20) / 800 ) * G         #G is Solar insolation or solar irradiation
    return t_op

In [7]:
def T_op_min(T_amb_min, NOCT, G):
    t_op_min = T_amb_min + ((NOCT - 20) / 800 ) * G
    return t_op_min

In [8]:
def T_op_max(T_amb_max, NOCT, G):
    t_op_max = T_amb_max + ((NOCT - 20)/800) * G
    return t_op_max

In [9]:
def V_min_eff(V_mpp_stc, gamma_p, t_op_max, T_STC = 25):
    v_min_eff = V_mpp_stc - ((gamma_p / 100) * (t_op_max - T_STC))
    return v_min_eff

In [10]:
def V_max_eff(V_oc_stc, gamma_V_oc, t_op_min, T_STC = 25):
    v_max_eff = V_oc_stc - ((gamma_V_oc / 100) * (t_op_min - T_STC))
    return v_max_eff

In [11]:
def M_string_min(V_inv_dc_min, V_min_eff):
    m_string_min = V_inv_dc_min / V_min_eff
    return math.ceil(m_string_min)

In [12]:
def M_string_max(V_inv_dc_max, V_max_eff):
    m_string_max = V_inv_dc_max / V_max_eff
    return math.ceil(m_string_max)

In [13]:
def f(*derating_factors):               #Takes percentage as input
    product = 1                         #100% is 1 and 50 is 0.5   
    for i in derating_factors:
        product *= int(i)
    return product

In [14]:
def I_sc_eff(I_sc_stc, gamma_I_sc, t_op_max, T_STC = 25):
    i_sc_eff = I_sc_stc - ((gamma_I_sc / 100) * (t_op_max - T_STC))
    return i_sc_eff

In [15]:
def S_max(I_inv_dc, i_sc_eff):
    return round(I_inv_dc / i_sc_eff)

In [16]:
def Series():
    T_OP = T_op(T_amb, NOCT, G) 
    #print(T_OP)
    T_OP_MIN = T_op_min(T_amb_min, NOCT, G)
    #print(T_OP_MIN)
    T_OP_MAX = T_op_max(T_amb_max, NOCT, G)
    #print(T_OP_MAX)
    
    V_MIN_EFF = V_max_eff(V_mpp_stc, gamma_p, T_OP_MAX)
    V_MAX_EFF = V_max_eff(V_oc_stc, gamma_v_oc, T_OP_MIN)
    
    M_STRING_MIN = M_string_min(V_inv_dc_min, V_MIN_EFF)
    #print(M_STRING_MIN)
    M_STRING_MAX = M_string_max(V_inv_dc_max, V_MAX_EFF)
    #print(M_STRING_MAX)
    
    I_SC_EFF = I_sc_eff(I_sc_stc, gamma_I_sc, T_OP_MAX)
    
    S_MAX = S_max(I_inv_dc, I_SC_EFF)
    
    return ((M_STRING_MIN, M_STRING_MAX), S_MAX)

In [17]:
def possible_configurations(config, inv_power_in_kw):
    series, parallel = config
    min_series, max_series = series
    
    possibilities = []
    
    for i in range(min_series, max_series + 1):
        temp = math.floor(inv_power_in_kw/(i * power_of_solar_panel))
        if temp <= parallel :
            possibilities.append((i, temp))
            
    return possibilities

In [18]:
def power_of_inverter(v,i):
    return v*i

In [19]:
def sorted_config(possibilities):
    no_of_panels = [(a*b,(a,b)) for a,b in possibilities]
    no_of_panels.sort()
    return no_of_panels

In [20]:
def number_of_inverters_required(desired_output, power_of_inverter):
    return math.ceil(desired_output/power_of_inverter)

In [21]:
def number_of_panels_required(no_of_panels_for_one_inverter, no_of_inverters):
    return no_of_panels_for_one_inverter * no_of_inverters

In [22]:
def Total_cost_of_project(total_number_of_panels, cost_of_one_panel, total_number_of_inverters, cost_of_one_inverter):
    cost_of_panels = total_number_of_panels * cost_of_one_panel
    cost_of_inveters = total_number_of_inverters * cost_of_one_inverter
    return cost_of_panels + cost_of_inveters

Input
====

In [23]:
config = Series()
print(config)

((12, 22), 69)


In [24]:
possibilities = possible_configurations(config, power_of_inverter(700, 600))
print(possibilities)

[(19, 69), (20, 65), (21, 62), (22, 59)]


In [25]:
sorted_possibilites = sorted_config(possibilities)
print(sorted_possibilites)

[(1298, (22, 59)), (1300, (20, 65)), (1302, (21, 62)), (1311, (19, 69))]


In [26]:
inverters = number_of_inverters_required(2500000, power_of_inverter(700, 600))
panels = number_of_panels_required(sorted_possibilites[0][0], inverters)

In [30]:
panels

7788

In [31]:
sorted_possibilites

[(1298, (22, 59)), (1300, (20, 65)), (1302, (21, 62)), (1311, (19, 69))]

In [27]:
Total_cost_of_project(panels, 7000, inverters, 20000)

54636000

Testing
======

In [28]:
Series()

((12, 22), 69)

In [29]:
22*69*350

531300

In [32]:
x = possible_configurations(((12,22), 69), 250000)

59
54
51
47
44
42
39
37
35
34
32


In [30]:
12*59*350

247800

In [31]:
13*54*350

245700

In [38]:
no_of_panels = [a*b for idx,(a,b) in enumerate(x)]

In [39]:
min(no_of_panels)

700

In [41]:
no_of_panels

[708, 702, 714, 705, 704, 714, 702, 703, 700, 714, 704]

In [43]:
no_of_panels.index(700)

8

In [44]:
x[8]

(20, 35)